## Assignment: Segmenting and Clustering Neighborhoods in Toronto

For this assignment, it is required to explore and cluster the neighborhoods in Toronto. So, let's start by creating a new Notebook for this assignment.

I will use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

### Step 1: Import Libraries ###

In [4]:
!pip install beautifulsoup4 -U
!pip install lxml -U
!pip install requests -U
!pip install html5lib -U
!pip install urllib3 -U
!pip install geocoder -U

     |████████████████████████████████| 102kB 15.4MB/s ta 0:00:01
  Found existing installation: beautifulsoup4 4.7.1
    Uninstalling beautifulsoup4-4.7.1:
      Successfully uninstalled beautifulsoup4-4.7.1
     |████████████████████████████████| 5.8MB 12.3MB/s eta 0:00:01
  Found existing installation: lxml 4.3.1
    Uninstalling lxml-4.3.1:
      Successfully uninstalled lxml-4.3.1
     |████████████████████████████████| 61kB 16.8MB/s eta 0:00:01
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0
Requirement already up-to-date: html5lib in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.0.1)
     |████████████████████████████████| 133kB 19.8MB/s eta 0:00:01
ERROR: ibm-cos-sdk-core 2.4.3 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.7 which is incompatible.
ERROR: botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.7 which is incompatible.
  Found e

In [5]:
import html.parser
from bs4 import BeautifulSoup
import requests
import urllib3.request
import pandas as pd
import numpy as np

### Step 2: Scraping the table from wikipedia ###

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(response, 'lxml')
table = soup.find('div', id = 'mw-content-text')
rows = table.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]
df = pd.DataFrame(columns=columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]
    else:
        pass
    df = df.append(pd.Series(values, index = columns), ignore_index=True)
    df.to_csv('toronto.csv')
df = pd.read_csv('toronto.csv')
df.shape

(292, 4)

In [7]:
df.head(10)

,Unnamed: 0,Postcode,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,Harbourfront
5,5,M6A,North York,Lawrence Heights
6,6,M6A,North York,Lawrence Manor
7,7,M7A,Queen's Park,Not assigned
8,8,M8A,Not assigned,Not assigned
9,9,M9A,Queen's Park,Queen's Park


### Step 3: Create DataFrame ###

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. Only to process the cells that have an assigned borough and ignore cells with a borough that is Not assigned below code has been used.

In [8]:
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df['Borough'].replace('Not assigned',np.nan,inplace = True)
df.dropna(subset=['Borough'], axis = 0, inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()
df_grouped = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.tail(20)
for index, row in df_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
df_grouped

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df_grouped.shape

(103, 3)

### Step 4: Use the csv file to create DataFrame

In [10]:
df_grouped.shape
!wget -q -O 'toronto_coord.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
toronto_coord = pd.read_csv("toronto_coord.csv")
toronto_coord.head()
toronto_coord.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
toronto_coord.head()

Data downloaded!


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
toronto_tab = df_grouped.merge(toronto_coord, on="PostalCode", how="left")
toronto_tab.head()
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_tab['Borough'].unique()),
        toronto_tab.shape[0]))
toronto_tab

The dataframe has 11 boroughs and 103 neighborhoods.


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Step 5: Clustering the neighborhoods in Toronto

I will be using the Foursquare API to explore neighborhoods in selected cities in Toronto. The Foursquare explore function will be used to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. Here I will be using *k*-means clustering for analysising the data of neighborhoods in Toronto. Finally, use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters. For that I will download the required libraries and dependencies.

In [12]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [13]:
toronto_tab.to_csv('toronto_tab.csv')
toronto_tab_csv = "toronto_tab.csv"

In [16]:
toronto_neighborhoods = pd.read_csv(toronto_tab_csv)
toronto_neighborhoods.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Toronto's latitude and longtitude are manually extracted via google search.

In [21]:
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto) 
map_toronto

### Let's simplify the above map and segment and cluster only the neighborhoods in Scarborough. So let's slice the original dataframe and create a new dataframe of the Scarborough data.

In [22]:
CLIENT_ID = '5CGGZXVKML4QGFGWGITB234HSQQCLIBJHP4QHDNBCKJCYLQX' 
CLIENT_SECRET = 'BWBCFYUBB4RFSHXM2W0QCMKPA3UWFGBLAGQSI30EMMDSRMZF'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5CGGZXVKML4QGFGWGITB234HSQQCLIBJHP4QHDNBCKJCYLQX
CLIENT_SECRET:BWBCFYUBB4RFSHXM2W0QCMKPA3UWFGBLAGQSI30EMMDSRMZF


In [23]:
scarborough_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Let's get the top 100 venues in the neighborhood 'Steeles West', from Scarborough within 500 meters radius

In [25]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude']
neighborhood_longitude = scarborough_data.loc[0, 'Longitude']

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood']

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.8066863, -79.19435340000003.


In [26]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ddcd5ebedbcad001c68393d'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [28]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
import json 
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
4,Hot Topic,Clothing Store,43.775450,-79.257929
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
9,Jimmy The Greek,Greek Restaurant,43.775112,-79.257119


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


### Let's explore Neighborhoods in Scarborough

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


### These are the venues for each Neighborhood in Scarborough

In [34]:
scarborough_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection


In [35]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


In [36]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 50 uniques categories.


### Analyze Each Neighborhood

In [40]:
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
scarb_onehot.shape

(86, 51)

### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000000,0.000,0.0,0.25,0.000000,0.000000,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.000,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.00,0.000000,0.0,0.000,0.0,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.25,0.000,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.000,0.0,0.000,0.25,0.0,0.000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.000000,0.0,0.000,0.0,0.0
3,Cedarbrae,0.0,0.125,0.0,0.125000,0.125,0.0,0.00,0.000000,0.000000,0.00,0.125,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.125,0.0,0.125,0.00,0.0,0.125,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.125,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000,0.0,0.222222,0.000,0.0,0.00,0.222222,0.111111,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.111111,0.000,0.0,0.000,0.00,0.0,0.000,0.0,0.111111,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.111111,0.0,0.000,0.0,0.0


### Get top 10 venues per neighborhood

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

In [47]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,Coffee Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Hakka Restaurant,College Stadium,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Park,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Coffee Shop,Gas Station,Furniture / Home Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Chinese Restaurant,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Fast Food Restaurant,Pharmacy,Noodle House,Vietnamese Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Coffee Shop,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Thrift / Vintage Store,Light Rail Station,Vietnamese Restaurant,Sandwich Place,Rental Car Location,Furniture / Home Store,Fried Chicken Joint
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Playground,Department Store,Coffee Shop,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
9,"Guildwood, Morningside, West Hill",Spa,Intersection,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop


### Cluster Neighborhoods:

#### Run k-means to cluster the neighborhood into 5 clusters.

In [61]:
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)

kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 0, 1, 0, 4, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [67]:
scarb_merged = scarb_data

scarb_merged['Cluster Labels'] = kmeans.labels_

scarb_merged

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0
5,5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0
6,6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,1
7,7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0
8,8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,4
9,9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0


In [68]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of the five clusters

In [69]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels
0,M1B,-79.194353,0
2,M1E,-79.188711,0
3,M1G,-79.216917,0
4,M1H,-79.239476,0
5,M1J,-79.239476,0
7,M1L,-79.284577,0
9,M1N,-79.264848,0
11,M1R,-79.295849,0
12,M1S,-79.262029,0
15,M1W,-79.318389,0


In [70]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels
6,M1K,-79.262029,1


In [71]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels
10,M1P,-79.273304,2


In [72]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels
13,M1T,-79.304302,3


In [73]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels
1,M1C,-79.160497,4
8,M1M,-79.239476,4
14,M1V,-79.284577,4
